<a href="https://colab.research.google.com/github/informatics-isi-edu/eye-ai-tools/blob/2-golden-standard-generation-for-diagnosis/notebooks/Golden_Label.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Prerequisites
!pip install git+https://github.com/fair-research/bdbag
!pip install deriva
!pip install 'git+https://github.com/informatics-isi-edu/eye-ai-tools.git@2-golden-standard-generation-for-diagnosis'


  Cloning https://github.com/fair-research/bdbag to /tmp/pip-req-build-f3xx38zv
  Running command git clone --filter=blob:none --quiet https://github.com/fair-research/bdbag /tmp/pip-req-build-f3xx38zv
  Resolved https://github.com/fair-research/bdbag to commit 8ee285059caa71a0789179b8d57b03010ac00e7f
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/informatics-isi-edu/eye-ai-tools.git (to revision 2-golden-standard-generation-for-diagnosis) to /tmp/pip-req-build-nh187f92
  Running command git clone --filter=blob:none --quiet https://github.com/informatics-isi-edu/eye-ai-tools.git /tmp/pip-req-build-nh187f92
  Running command git checkout -b 2-golden-standard-generation-for-diagnosis --track origin/2-golden-standard-generation-for-diagnosis
  Switched to a new branch '2-golden-standard-generation-for-diagnosis'
  Branch '2-golden-standard-generation-for-diagnosis' set up to track remote branch '2-golden-standard-generation-for-diagnosis' from 'origin'.
  Resolved h

In [2]:
# login to DERIVA via Globus Auth
host = 'dev.eye-ai.org' #@param {type:"string"}
!deriva-globus-auth-utils login --no-browser --host {host}

You are already logged in.


Make a connection to the Eye AI catalog and get a reference to the main schema: eye-ai

In [3]:
# @title Configure Grader groups
grader_list = ["Benjamin Xu", "Kendra Hong", "Kent Nguyen"] # @param
dataset_rid = "2-277M" # @param {type:"string"}
diagnosis_tag_rid = "2-35RM" # @param {type:"string"}


In [4]:
# @title Configure Process and
process_tag_name = "Test Diagnosis Generation" # @param {type:"string"}
Metadata = "Test Diagnosis Generation" # @param {type:"string"}
Github_URL="" # @param {type:"string"}
Process_Tag = "" # @param {type: "string"}
Description = "" # @param {type: "string"}
Github_Checksum = "" # @param {type: "string"}



Diagnosis_Tag_name = "Test Diagnosis Generation" # @param {type:"string"}
Description = "Generate new diagnosis based on a group of graders: "+str(grader_list)
synonyms = [] # @param


In [5]:
from ml_utils import EyeAI
EA = EyeAI(hostname = 'dev.eye-ai.org', catalog_number = 'eye-ai')

In [6]:
# @title Define generation rule
def CDR_rule(values):
    return values.mean()

def Diag_rule(values):
    # Use the mode function to get the most frequent diagnosis value
    mode_diagnosis = values.mode()

    # If there are multiple modes, take the first one
    if not mode_diagnosis.empty:
        return mode_diagnosis.iloc[0]
    else:
        # Handle the case where there is no mode (e.g., all values are unique)
        return 'suspect/Not'

def ImageQuality_rule(values):
    mode_quality = values.mode()
    if not mode_quality.empty:
        return mode_quality.iloc[0]
    else:
        return "Good/Bad"

In [7]:
# @title Extract the diagnosis in the catalog and generate new Diagnosis
all_diag = EA.image_tall(dataset_rid, diagnosis_tag_rid)
result = EA.generate_Diagnosis(all_diag[all_diag["Full_Name"].isin(grader_list)], Diag_rule, CDR_rule, ImageQuality_rule)

In [9]:
# @title Insert New Process_Tag, Proecess, and Diagnosis_Tag
new_process_tag_RID = EA.insert_new_process(table_name='Process_Tag', name=process_tag_name, description=grader_list, synonyms=[], exist_ok=False)
Process_RID = EA.insert_new_process(Metadata, Github_URL, new_process_tag_RID, Description, Github_Checksum)

new_diag_tag_RID = EA.insert_new_tag('Diagnosis_Tag', Diagnosis_Tag_name, Description, exist_ok=False)


In [ ]:
entities = EA.insert_new_diag(result[:5], new_diag_tag_RID, Process_RID)

0 1
1 2
2 3
3 4
4 5
